#<h1><center><b> Apple Disease MobileNet Hyperparameter </b></center></h1>

## Import

In [ ]:
# from google.colab import files

import os, sys, pdb, shutil, time, random
import argparse
import torch
import torch.backends.cudnn as cudnn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as func
import torchvision
from torchvision import transforms, datasets, models
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from shutil import copyfile
from os import listdir, makedirs, getcwd, remove
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np

import torch.nn as nn
from torch.optim.lr_scheduler import MultiStepLR
from torchvision.utils import make_grid
from skimage import io
# import splitfolders as sp
from torch.optim import Adam
from google.colab import files

In [ ]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11687 sha256=f1feb2220bf5ee3141a1c77db047c60d363255de603e56a9af97ced402c8fb17
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


## 1.Download Dataset

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d akshaygudi/apple-leaf-disease

 98% 794M/813M [00:08<00:00, 103MB/s]
100% 813M/813M [00:08<00:00, 105MB/s]


In [ ]:
from zipfile import ZipFile
filename = "apple-leaf-disease.zip"

with ZipFile(filename,'r') as zip_dir:
    zip_dir.extractall()
    print("Done")

Done


##<h2>2. Custom Dataset<h2>

In [ ]:
class MyDataset(Dataset):
    
    def __init__(self,csv_file,root_dir,transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        img_path = os.path.join(self.root_dir,self.annotations.iloc[index,0])
#         print(img_path)
#         image = io.imread(img_path)
        image = Image.open(img_path).convert('RGB')
        ylabel = torch.tensor(int(self.annotations.iloc[index,-1]))
        
        if self.transform:
            image = self.transform(image)
        
        return (image,ylabel)

## <h2>3. Initialization </h2>

In [ ]:
args = {}

args["dataset"] = "apple"
args["model"] = "resnet18"
args["num_classes"] =  4
args["batch_size"] = 16
args["epochs"] = 50
args["learning_rate"] = 0.001
args["data_augmentation"] = False
args["cutout"] = True
args["n_holes"] = 1
args["length"] = 20
args["no_cuda"] = False
args["seed"] = 0

args["cuda"] = not args["no_cuda"] and torch.cuda.is_available()
cudnn.benchmark = True  

torch.manual_seed(args["seed"])
if args["cuda"]:
    torch.cuda.manual_seed(args["seed"])

test_id = args["dataset"] + '_' + args["model"]

print(args)

{'dataset': 'apple', 'model': 'resnet18', 'num_classes': 4, 'batch_size': 16, 'epochs': 50, 'learning_rate': 0.001, 'data_augmentation': False, 'cutout': True, 'n_holes': 1, 'length': 20, 'no_cuda': False, 'seed': 0, 'cuda': True}


## <h2>4. DataLoader and Transforms<h2>

### <h3>4.1 Cutout Regularization</h3>

In [ ]:
class Cutout(object):
    """Randomly mask out one or more patches from an image.

    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask
        
        return img

### <h3>4.2 Transform for Train Dataset</h3>

In [ ]:
def get_train_transform(cutout_length):
  normalize_train = transforms.Normalize(mean = [0.4066, 0.5148, 0.3226],
                                      std=[0.2011, 0.1875, 0.1882])

  train_transform = transforms.Compose([])

  if args["data_augmentation"]:
      train_transform.transforms.append(transforms.RandomHorizontalFlip())

  train_transform.transforms.append(transforms.Resize((224,224)))
  train_transform.transforms.append(transforms.ToTensor())
  train_transform.transforms.append(normalize_train)


  if args["cutout"]:
      train_transform.transforms.append(Cutout(n_holes=args["n_holes"], length=cutout_length))

  return train_transform


In [ ]:
train_trans = get_train_transform(args["length"])

### <h3>4.3 Transform for Test Dataset</h3>

In [ ]:
def get_test_transform():
  normalize_test = transforms.Normalize(mean = [0.4047, 0.5141, 0.3240],
                                     std=[0.2019, 0.1883, 0.1881])

  test_transform = transforms.Compose([])
  test_transform.transforms.append(transforms.Resize((224,224)))
  test_transform.transforms.append(transforms.ToTensor())
  test_transform.transforms.append(normalize_test)

  return test_transform

In [ ]:
valid_trans = get_test_transform()

### <h3>4.4 Create Data Loader</h3>

In [ ]:
train_set = MyDataset(csv_file="Apple_Leaf_Disease/train_data.csv",root_dir="Apple_Leaf_Disease/images/",transform=train_trans)
valid_set = MyDataset(csv_file="Apple_Leaf_Disease/test_data.csv",root_dir="Apple_Leaf_Disease/images/",transform=valid_trans)
        
train_loader = DataLoader(train_set, batch_size=args["batch_size"],shuffle=True)
test_loader = DataLoader(valid_set, batch_size=args["batch_size"], shuffle=False)

dataset_sizes = {
    'train': len(train_loader.dataset), 
    'valid': len(test_loader.dataset)
}

## <h2>5. MODELS</h2>

### <h3>5.1 Test Resnet working</h3>

In [ ]:
use_gpu = torch.cuda.is_available()

mobilenet_v2_model = models.mobilenet_v2(pretrained=True)
inputs, labels = next(iter(test_loader))
if use_gpu:
    mobilenet_v2_model = mobilenet_v2_model.cuda()
    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())   
else:
    inputs, labels = Variable(inputs), Variable(labels)
outputs = mobilenet_v2_model(inputs)
outputs.size()

torch.Size([16, 1000])

### <h3>5.2 Create Model</h3>

In [ ]:
cnn = models.mobilenet_v2(pretrained=True)
args["num_classes"] = 4
# freeze all model parameters
for param in cnn.parameters():
    param.requires_grad = False

# New final layer with 4 classes
num_ftrs = cnn.classifier[1].in_features
cnn.classifier = torch.nn.Linear(num_ftrs, args["num_classes"])

if use_gpu:
    cnn = cnn.cuda()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

bs = 64
test_dl = DataLoader(valid_set, batch_size=bs,shuffle=False)

print("Done - Configure Model")

Done - Configure Model


## <h2>6. Unfreeze</h2>

In [ ]:
for name, child in cnn.named_children():
    if name in ['features', 'classifier']:
        print(name + ' is unfrozen')
        for param in child.parameters():
            param.requires_grad = True
    else:
        print(name + ' is frozen')
        for param in child.parameters():
            param.requires_grad = False

features is unfrozen
classifier is unfrozen


## 7.Bayesian Optimization

In [ ]:
loss_func = nn.CrossEntropyLoss().cuda()
# cnn_optimizer = torch.optim.SGD(cnn.parameters(), lr=args["learning_rate"],
#                                 momentum=0.9, nesterov=True, weight_decay=5e-4)

def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [ ]:
def get_model_accuracy():
    tot_acc = 0
    avg_acc = 0
  
    # Set model to evaluation mode
    cnn.eval()

    for xbt, ybt in test_dl:

        pred = cnn(xbt.to(device))
        tot_acc += accuracy(pred,ybt.to(device))

    avg_acc = tot_acc / len(test_dl)
  
    return avg_acc.item()


# Print accuracy of model
# print("The average accuracy is: " + str(get_model_accuracy()))

In [ ]:
def obj_func(cutout_len):
      
      # lr, bs, epochs,
      # We need to round off bs and epochs because Gaussian processes cannot deal with discrete variables 
      # cutout_len
    bs = int(36)
    epochs = int(18)

    train_trans = get_train_transform(int(cutout_len))

    train_set = MyDataset(csv_file="imag_data/train_data.csv",root_dir="imag_data/images",transform=train_trans)
    valid_set = MyDataset(csv_file="imag_data/test_data.csv",root_dir="imag_data/images",transform=valid_trans)

    train_dl = DataLoader(train_set, batch_size=bs,shuffle=True)
    test_dl = DataLoader(valid_set, batch_size=bs, shuffle=False)
      
    optim = Adam(cnn.parameters(), lr = 0.0001)
      
    for epoch in range(epochs):
    
        for xb, yb in train_dl:
        
            # .to(device) moves torch.Tensor objects to the GPU for faster training
        
            preds = cnn(xb.to(device))
            loss = loss_func(preds, yb.to(device))
            acc = accuracy(preds,yb.to(device))
        
            loss.backward()
            optim.step()
            optim.zero_grad()
        
        print("Train Loss: " + str(loss.item()) + "\t \t Train Accuracy: " + str(100 * acc.item()))

    acc = get_model_accuracy()
      
    return acc

In [ ]:
from bayes_opt import BayesianOptimization
# 'cutout_len':(16,18) 
# Bounded region of parameter space
pbounds = {'lr': (1e-4,1e-2), 'bs': (16,64), 'epochs': (1,30)}

optimizer = BayesianOptimization(
    f=obj_func,
    pbounds=pbounds,
    verbose=2, # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)

optimizer.maximize(init_points=2, n_iter=3)

print(optimizer.max)

|   iter    |  target   | cutout... |
-------------------------------------
Train Loss: 0.2704342305660248	 	 Train Accuracy: 93.1034505367279
Train Loss: 0.2779194712638855	 	 Train Accuracy: 86.20689511299133
Train Loss: 0.11967669427394867	 	 Train Accuracy: 93.1034505367279
Train Loss: 0.027716679498553276	 	 Train Accuracy: 100.0
Train Loss: 0.01673622988164425	 	 Train Accuracy: 100.0
Train Loss: 0.004650220740586519	 	 Train Accuracy: 100.0
Train Loss: 0.03777804598212242	 	 Train Accuracy: 100.0
Train Loss: 0.005007901228964329	 	 Train Accuracy: 100.0
Train Loss: 0.09169457852840424	 	 Train Accuracy: 96.55172228813171
Train Loss: 0.027327848598361015	 	 Train Accuracy: 100.0
Train Loss: 0.0006421500584110618	 	 Train Accuracy: 100.0
Train Loss: 0.09783957898616791	 	 Train Accuracy: 96.55172228813171
Train Loss: 0.014129611663520336	 	 Train Accuracy: 100.0
Train Loss: 0.048511140048503876	 	 Train Accuracy: 96.55172228813171
Train Loss: 0.0031522237695753574	 	 Train Accurac